# Getting started with the Tuna backends

**Author:** Marios Samiotis (m.samiotis@tudelft.nl)

**Date:** December 19, 2025

# 1. Introduction

In this notebook we will get started with using the Tuna superconducting quantum backends which exist at the [Quantum Inspire 2.0](https://www.quantum-inspire.com/) cloud services!

Prior to using this notebook, for optimal performance, make sure that you are running it within a customized Python 3.12 environment which includes the packages "quantuminspire" and "qiskit-quantuminspire".

For details on how to create such a Python environment, follow the installation instructions in the [README file](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/README.md).

Useful links:
1. [Tuna Backends Operational Specifics](https://www.quantum-inspire.com/kbase/tuna-operational-specifics/)
2. [Backend Performance Dashboards](https://monitoring.qutech.support/public-dashboards/c494a21fb6b7405f850ab8f340f798ef?orgId=1&refresh=10s) [live updates]
3. [Join our Slack community!](https://join.slack.com/t/qisuperconducting/shared_invite/zt-35o7zitdh-_9QPmB53hhLy12Eat5gwWA)

First, we run the following cell in order to login to the Quantum Inspire platform. You need an account in order to login to the platform.

Please click on "Confirm" in the pop-up browser window to login.

In [1]:
! qi login "https://api.quantum-inspire.com"

Please continue logging in by opening: https://auth.quantum-inspire.com/activate?user_code=LTBD-TRPM in your browser
If promped to verify a code, please confirm it is as follows: LTBD-TRPM
Login successful!
Using member ID 10


Run the following cell for all necessary library imports,

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from qiskit import QuantumCircuit, transpile
from qiskit_quantuminspire.qi_provider import QIProvider
from qi_utilities.utility_functions.circuit_modifiers import apply_readout_circuit
from qi_utilities.utility_functions.raw_data_processing import get_multi_counts, get_multi_probs
from qi_utilities.utility_functions.readout_correction import (split_raw_shots, extract_ro_assignment_matrix, plot_ro_assignment_matrix,
                                                               get_ro_corrected_multi_probs)
from qi_utilities.utility_functions.data_handling import StoreProjectRecord, RetrieveProjectRecord

# 2. Connecting to the backend

We first connect to the Quantum Inspire provider, by running the following cell,

In [ ]:
provider = QIProvider()

The following command prints the full list of all available backends of the platform,

In [ ]:
provider.backends()

We now connect to a Tuna backend by running the following cell,

In [ ]:
backend_name = "Tuna-9"
backend = provider.get_backend(name=backend_name)

By running the cell below, the connectivity of the processor is shown (otherwise known as "coupling map"),

In [ ]:
backend.coupling_map.draw()

Any single- or multi-qubit gates that may be applied on the qubits of the Tuna backends are referred to as "instructions". The full list of allowed instructions of the Tuna backend is shown by running the following cell,

In [ ]:
backend.instructions

When executing a quantum algorithm on any backend, one needs to choose the number of measurement shots to be performed for the given quantum circuit. The maximum number of allowed shots on the Tuna backend can be obtained by running the command

In [ ]:
backend.max_shots

# 3. Running your first quantum circuit on a Tuna backend

In this section, we create a quantum circuit which generates an entangled two-qubit quantum state, better known as the [Bell-pair state](https://en.wikipedia.org/wiki/Bell_state).
For more information concerning what a Bell-pair state is, [click here](https://www.quantum-inspire.com/kbase/hello-quantum-world/).

To create a Bell-pair state on a Tuna backend, one needs to apply a two-qubit gate between two qubits that are directly coupled to each other. Such a circuit can be created in two ways: either by taking into consideration the number of qubits and connectivity of the device when writing the code, or by creating a theoretical circuit consisting of two qubits and then using the Qiskit transpiler, which subsequently maps the operations according to the connectivity and qubit number constraints.

In Section 3.1 we create a quantum circuit with a quantum register of size the total number of qubits of the Tuna backend, respecting the connectivity of the device, while in Section 3.2 we use the Qiskit transpiler, and perform post-processing on the measurement outcomes to mitigate readout errors which occur during measurement.

## 3.1. Creating a Bell-pair state without the use of the Qiskit transpiler

Since we are not using the Qiskit transpiler, we have to define a quantum circuit with a number of qubits equal to the total number of qubits of the Tuna backend, as well as perform two-qubit gates on qubits that are coupled together (see the coupling map of the processor above).

In addition, one is advised to consult the appropriate Tuna backend [performance dashboard](https://monitoring.qutech.support/public-dashboards/c494a21fb6b7405f850ab8f340f798ef?orgId=1&refresh=10s) with choosing the most performant two-qubit pair at a given moment in time. At the time of writing this guide, qubits Q0 and Q2 seemed to be the best choice.

In [ ]:
qubit_0 = 0 # the entry "0" is for qubit "Q0"
qubit_1 = 2 # the entry "2" is for qubit "Q2"

qc = QuantumCircuit(9, # total number of qubits of the Tuna backend
                    2, # classical bit register size
                    name = f'Bell_State_Q{qubit_0}_Q{qubit_1}') # by extension, the project name

qc.h(qubit_0)
qc.cx(qubit_0, qubit_1)
qc.measure(qubit_0, cbit=0)
qc.measure(qubit_1, cbit=1)

We may visualize the above quantum circuit by running the following cell,

In [ ]:
qc.draw('mpl')

Notice the bit-ordering convention being used here. For a classical bit register of size N, and a classical bit cbit_i where i is the bit index, the measurement results are always stored as ["cbit_N-1 cbit_N-2 ... cbit_1 cbit_0"].

Now we run the above circuit on the Tuna backend. We choose for the number of circuit shots the maximum allowed, which as we have already seen in Section 2 is given by "backend.max_shots".

Usually for regular-sized jobs and a normal queue, the Bell-state algorithm should take only a few seconds of run time, and the result should be retrieved within a minute. Nevertheless, it is a safe practice to request a timeout of about 10 minutes for an unusually busy queue.

In [ ]:
nr_shots = backend.max_shots
job = backend.run(qc, shots=nr_shots)
result = job.result(timeout = 600) # timeout is in units of seconds

We store the job (project) record in a local directory inside the local user 'Documents' folder by running,

In [ ]:
StoreProjectRecord(job)

We can retrieve the results from our local record by running the following cell,

In [ ]:
job_id = '346233' # enter the Job ID

loaded_result = RetrieveProjectRecord(job_id)

We now gather the measurement counts,

In [ ]:
counts = result.get_counts()

Notice how, if successfully loaded, the counts from 'loaded_result' and 'result' objects have the same structure,

In [ ]:
result.get_counts() == loaded_result.get_counts()

Finally, we visualize the results,

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)

for bit_string in counts:
    ax.bar(bit_string,
           counts[bit_string] / nr_shots,
           color='blue', alpha=0.5)

ax.set_xlabel("Bit strings")
ax.set_ylabel("Probabilities")
ax.set_title(f"Bell-state preparation\n{backend_name} processor\nQubits Q{qubit_0} - Q{qubit_1}")

ax.set_yticks([0.0, 0.25, 0.5, 0.75, 1.0])
ax.set_ylim(0.0, 1.05)

plt.grid(axis='y')
plt.show()

## 3.2. Creating a Bell-pair state by using the Qiskit transpiler and applying readout error mitigation (ADVANCED)

In this section, we will be using Qiskit's "transpile" function, which simplifies circuit writing since the user does not need to consider the number of qubits, the coupling map or the allowed set of instructions of the Tuna backends when creating a circuit; the transpiler takes care of all that. Additionally, we will be post-processing the raw data after receiving the job result, in order to mitigate readout errors that occur during the measurement.

In the previous example, the quantum circuit that we created uses a CNOT gate between two qubits. Reading through the Tuna backends [operational specifics](https://www.quantum-inspire.com/kbase/tuna-operational-specifics/), one notices that the only native two-qubit gate in the Tuna backends is the CZ gate. How can it be that the CNOT gate was being executed by the Tuna backend in the previous example?

Whenever a user sends a quantum circuit to the Tuna backends which includes a CNOT gate, or any other non-native two-qubit gate, those are being decomposed by the Tuna backend internal compilation software into a combination of single-qubit gates and CZ gates, an internal transpilation step which is not visible to the user.

If the user is concerned with optimizing their circuits so that those do not get further decomposed by the Tuna backend internal compilation software, then they need to define the following basis gates list which correspond to the accurate set of allowed instructions of the device, which is then passed to the Qiskit transpiler manually,

In [ ]:
tuna_backends_basis_gates = ['id', 's', 'sdg', 't', 'tdg', 'x', 'rx', 'y', 'ry', 'z', 'rz', 'cz', 'delay', 'reset']

Now let us define once again the Bell-pair state preparation quantum circuit. Using the mid-circuit measurement functionality (more on that in Section 4 of this notebook), we append the necessary measurements, used later for mitigating readout errors, at the end of the original circuit,

In [ ]:
# This is a 'theoretical' quantum circuit: the qubit number as well
# as the connectivity of the Tuna backend is being ignored

qubit_0 = 0
qubit_1 = 1

qc = QuantumCircuit(2, 2, name=f'Bell_State')

qc.h(qubit_0)
qc.cx(qubit_0, qubit_1)
qc.measure(qubit_0, cbit=0)
qc.measure(qubit_1, cbit=1)

qc = apply_readout_circuit(qc, [qubit_0, qubit_1])

In [ ]:
qc.draw('mpl')

In order to run the above circuit on the Tuna backend, it first needs to be transpiled.

But how does the transpiler know which qubit pair to choose? If the user does not specify it, it begins with the first entry of the coupling map of the device.

Nevertheless, the user has the freedom to define a qubit priority list: by consulting the appropriate [performance dashboard](https://monitoring.qutech.support/public-dashboards/c494a21fb6b7405f850ab8f340f798ef?orgId=1&refresh=10s) page of the Tuna backend, one can create a list of qubits whose two-qubit gate performance is the most optimal at the given moment in time.

At the day of writing this guide, the best list was chosen as qubit_list = [0, 2]

In [ ]:
qubit_list = [0, 2]

In [ ]:
qc_transpiled = transpile(qc, backend, initial_layout=qubit_list, basis_gates=tuna_backends_basis_gates)

In [ ]:
qc_transpiled.draw('mpl')

As is shown in the above figure, the transpiler added additional "ancilla" qubits to the original circuit, which are called so since they are not being used in the algorithm.

In the above circuit, we see the further decomposition of the CNOT gate into single-qubit gates and a single CZ gate. This circuit will be executed by the Tuna backend as it is, without any further internal decomposition.

We now run the transpiled circuit on the Tuna backend,

In [ ]:
nr_shots = backend.max_shots
job = backend.run(qc_transpiled, shots=nr_shots, memory = True) # NOTE: memory is set to True in order to return raw data!
result = job.result(timeout = 600)

and store the project record locally. Notice how this time an HDF5 file is also created in the job folder, containing the raw data,

In [ ]:
StoreProjectRecord(job)

By retrieving the raw data from the project record, notice that the structure of the raw data in 'result' and 'loaded_result' is identical,

In [ ]:
job_id = '346234'

loaded_result = RetrieveProjectRecord(job_id)

In [ ]:
result.get_memory() == loaded_result.get_memory()

The following functions are used to separate the shots related to the Bell-state preparation from the shots related to constructing the readout assignment matrix, which is used in post processing to mitigate the effects of readout errors on the raw data.

In [ ]:
raw_data_shots, ro_mitigation_shots = split_raw_shots(result, qubit_list)
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, qubit_list)

raw_data_counts = get_multi_counts(raw_data_shots, len(qubit_list))
raw_data_probs = get_multi_probs(raw_data_counts)
ro_corrected_probs = get_ro_corrected_multi_probs(raw_data_probs, ro_assignment_matrix, qubit_list)

By plotting the readout assignment matrix, the user can gain valuable information about the state of the readout at the time of executing the quantum circuit on the device,

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, qubit_list)

Below we plot together the final results of the Bell-state preparation job with and without the use of readout error mitigation, 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)

bit_strings = [bit_string for bit_string in raw_data_probs[0]]

ax.bar(bit_strings,
       [raw_data_probs[0][bit_string] for bit_string in raw_data_probs[0]],
       color='blue',
       label='Raw data',
       alpha=0.5, align='edge', width=-0.4)
ax.bar(bit_strings,
       [ro_corrected_probs[0][bit_string] for bit_string in ro_corrected_probs[0]],
       color='orange',
       label='Readout-error-mitigation data',
       align='edge', width=0.4)

ax.set_xlabel("Bit strings")
ax.set_ylabel("Probabilities")
ax.set_title(f"Bell-state preparation\n{backend_name} processor\nQubits Q{qubit_list[0]} - Q{qubit_list[1]}")

ax.set_yticks([0.0, 0.25, 0.5, 0.75, 1.0])
ax.set_ylim(0.0, 1.05)
ax.legend()

plt.grid(axis='y')
plt.show()

# 4. Performing a qubit $T_1$ measurement using mid-circuit measurements

In this section, we perform a measurement used widely to estimate the relaxation time $T_1$ of a qubit, using the mid-circuit functionality of the Quantum Inspire SDK. For more information concerning what the qubit relaxation time is or what circuit routines are used to measure it, refer to [A Quantum Engineer's Guide to Superconducting Qubits](https://arxiv.org/abs/1904.06560v5), Section III. B. 2.

We first define the experiment parameters,

In [ ]:
qubit_nr = 0 # e.g. the entry "0" is for qubit "Q0"
total_time = 120e-6 # total measurement time in units of [s]
nr_points = 31 # number of measurement points

and then define the experiment circuit. The user does not need to alter the following code,

In [ ]:
cycle_time = 20e-9 # cycle time limitation imposed by the Tuna backend internal software
dt = total_time / nr_points
measurement_times = np.linspace(start = 0.0, stop = total_time, num = nr_points)

qc = QuantumCircuit(1, nr_points, name=f'T1_Q{qubit_nr}')
qc.x(qubit_nr) # qubit initialization to the |1> state
qc.measure(qubit = qubit_nr, cbit = 0) # initial measurement at time t=0
for time_idx in range(1, nr_points):
    qc.delay(duration = int((dt / cycle_time)), qarg = qubit_nr) # delay in units of cycle_time
    qc.measure(qubit = qubit_nr, cbit = time_idx)

qc = apply_readout_circuit(qc, [qubit_nr])

In [ ]:
qc.draw('mpl')

Let us now understand the above circuit. A $T_1$ measurement begins with an initial pi pulse ($X$ gate) on the measured qubit, bringing it to its excited state $|1\rangle$, followed by an idle time which we refer to as the "total_time", where we sample the qubit state for a number of "nr_points". An initially excited qubit $|1\rangle$ subsequently relaxes with a high probability to the ground state $|0\rangle$ within the time interval "total_time" defined above.

The "total_time" of the experiment is chosen to be approximately 4 times the qubit relaxation time $T_1$, which was already known at the time of writing this guide, thus we chose $120~\mu s$ as the total duration.

The circuit contains multiple measurement blocks equal to the "nr_points", with an additional two at the very end which are used for the purpose of constructing the readout assignment matrix. These blocks are what we refer to as "mid-circuit measurements".

How do the results of such a measurement look like? The measurement outcome of each measurement block is stored in a separate bit of the classical bit register, which has a total size equal to "nr_points" + 2.

Notice how in the method ".run()" we set "memory = True", in order to return the raw data of the experiment.

In [ ]:
nr_shots = 2**11 # NOTE: the user has to be careful with nr_shots using mid-circuit measurements
                 #       if the job fails to be executed, reduce the nr_shots
job = backend.run(qc, shots=nr_shots, memory = True) # NOTE: memory is set to True in order to return raw data!
result = job.result(timeout = 600)

In [ ]:
StoreProjectRecord(job)

Let us now investigate how the data is organized by first separating the raw data shots from those used to construct the readout assignment matrix,

In [ ]:
raw_data_shots, ro_mitigation_shots = split_raw_shots(result, [qubit_nr])

The raw_data_shots list has a total "nr_shots" entries, since we run this circuit for that particular number of shots, and each entry has "nr_points" bits, with each containing the result of each measurement block for the particular circuit run.

The right-most bit of each bit string corresponds to the first measurement block of the quantum circuit, while the left-most bit of each bit string corresponds to the final measurement.

In [ ]:
len(raw_data_shots) # size of the raw_data_shots list, equal to nr_shots

In [ ]:
len(raw_data_shots[0]) # size of the bit string of each circuit shot, equal to nr_points

In [ ]:
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, [qubit_nr])

raw_data_counts = get_multi_counts(raw_data_shots, len([qubit_nr]))
raw_data_probs = get_multi_probs(raw_data_counts)
ro_corrected_probs = get_ro_corrected_multi_probs(raw_data_probs, ro_assignment_matrix, [qubit_nr])

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, [qubit_nr])

Finally, we plot the probability of the qubit being in the state $|1\rangle$ for each measurement block, which decreases exponentially with time, eventually reaching the ground state $|0\rangle$. By fitting an exponential curve on the measurement data, we extract the relaxation time $T_1$ of the qubit,

In [ ]:
probabilities_excited = [ro_corrected_probs[entry]['1'] for entry in range(nr_points)]

def exponential_func(x, a, b):
    return a * np.exp(b*x)
params, covariance = curve_fit(exponential_func,
                               measurement_times,
                               probabilities_excited)
a_fit, b_fit = params
exponential_fit = exponential_func(measurement_times, a_fit, b_fit)

fig, ax = plt.subplots(dpi=300)
ax.scatter(1e6*measurement_times,
           probabilities_excited,
           label='Data', alpha=0.9)
ax.plot(1e6*measurement_times, exponential_fit, label='exp() fit', color='r')
ax.set_xlabel('Time (μs)')
ax.set_ylabel(r'$P(|1\rangle)$')
ax.set_title(f'T1 measurement\n{backend_name} processor\nQubit "Q{qubit_nr}"')

ax.text(x = 0.5*ax.get_xlim()[1], y = 0.95*ax.get_ylim()[1], s = f'T1 = {- 1e6 * 1 / b_fit:.1f} μs',
        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 5})
ax.set_ylim(-0.05, 1.05)

plt.legend()
plt.show()

# 5. Performing a Rabi oscillation using mid-circuit measurements

In order to perform a [Rabi oscillation](https://en.wikipedia.org/wiki/Rabi_cycle) on a qubit, we need to apply a transversal gate of varied amplitude for different measurement steps. A varied amplitude in our case translates to a varied angle of the applied single-qubit gate.

In a similar fashion to what has already been done in Section 4, we first define the experiment parameters,

In [ ]:
qubit_nr = 0 # e.g. the entry "0" is for qubit "Q0"
angle_step = np.pi / 14
total_steps = int(2*np.pi / angle_step) # 2*np.pi is used here for a full revolution

In [ ]:
qc = QuantumCircuit(1, total_steps, name=f'Rabi_Q{qubit_nr}')

for step_idx in range(total_steps):
    qc.delay(6000, qubit_nr) # initialization time for the qubit in units of CC cycles, each cycle worth 20 ns
    qc.rx(step_idx * angle_step, qubit_nr)
    qc.measure(qubit = qubit_nr, cbit = step_idx)

qc = apply_readout_circuit(qc, [qubit_nr])

In [ ]:
qc.draw('mpl')

Now let us run the above circuit. Notice how nr_shots has been set to $2^{12}$, since we are using multiple measurement blocks. If we request too many shots, we risk having the job fail due to memory limitations of the readout instruments.

In [ ]:
nr_shots = 2**12 # NOTE: the user has to be careful with nr_shots using mid-circuit measurements
                #       if the job fails to be executed, reduce the nr_shots
job = backend.run(qc, shots=nr_shots, memory = True)
result = job.result(timeout = 600)

In [ ]:
StoreProjectRecord(job)

We obtain the readout assignment matrix, as well as the probabilities for each measurement block, and visualize them

In [ ]:
raw_data_shots, ro_mitigation_shots = split_raw_shots(result, [qubit_nr])
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, [qubit_nr])

raw_data_counts = get_multi_counts(raw_data_shots, len([qubit_nr]))
raw_data_probs = get_multi_probs(raw_data_counts)
ro_corrected_probs = get_ro_corrected_multi_probs(raw_data_probs, ro_assignment_matrix, [qubit_nr])

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, [qubit_nr])

In [ ]:
probabilities_excited = [ro_corrected_probs[entry]['1'] for entry in range(total_steps)]

def cos_func(x, a, b, c, d):
    return a * np.cos(2*np.pi*b*x + c) + d
params, covariance = curve_fit(cos_func,
                               np.arange(0, total_steps, 1),
                               probabilities_excited)
a_fit, b_fit, c_fit, d_fit = params
cosine_fit = cos_func(np.arange(0, total_steps, 1), a_fit, b_fit, c_fit, d_fit)

fig, ax = plt.subplots(figsize=(18, 5), dpi=300)
ax.scatter(np.arange(0, total_steps, 1),
           probabilities_excited,
           label='Data', alpha=0.9)
ax.plot(np.arange(0, total_steps, 1), cosine_fit, label=r'$\cos()$' + ' fit', color='r')
ax.set_xlabel('Applied rotation')
ax.set_ylabel(r'$P(|1\rangle)$')
ax.set_title(f'Rabi oscillation\n{backend_name} processor\nQubit "Q{qubit_nr}"')

labels = []
for step_idx in range(total_steps):
    angle_in_degrees = (360 / (2 * np.pi)) * step_idx * angle_step
    step_label = f"rx{round(angle_in_degrees)}"
    labels.append(step_label)
label_locs = np.arange(0, total_steps, 1)
ax.set_xticks(label_locs)
ax.set_xticklabels(labels, rotation=65)
ax.set_ylim(-0.05, 1.05)

plt.grid()
plt.legend()
plt.show()

# SCRATCHPAD

In [ ]:
starmon_7_specs = {
    'Qubits':{
        'Q0': {
            'Frequency [Hz]': 6.251e9,
            'T1 [s]': 27.8e-6,
            'T2 [s]': 11.3e-6,
            'RB error': 6.614e-4,
            'SSRO': {
                'p0given1': 0.0334,
                'p1given0': 0.0049
            }},
        'Q1': {
            'Frequency [Hz]': 6.227e9,
            'T1 [s]': 11.9e-6,
            'T2 [s]': 6.5e-6,
            'RB error': 8.72e-4,
            'SSRO': {
                'p0given1': 0.0226,
                'p1given0': 0.0024
            }},
        'Q2': {
            'Frequency [Hz]': 5.590e9,
            'T1 [s]': 26.0e-6,
            'T2 [s]': 4.9e-6,
            'RB error': 0.00167,
            'SSRO': {
                'p0given1': 0.0179,
                'p1given0': 0.0111
            }},
        'Q3': {
            'Frequency [Hz]': 5.757e9,
            'T1 [s]': 26.2e-6,
            'T2 [s]': 13.1e-6,
            'RB error': 0.00167,
            'SSRO': {
                'p0given1': 0.0521,
                'p1given0': 0.0327
            }},
        'Q4': {
            'Frequency [Hz]': 5.753e9,
            'T1 [s]': 25.6e-6,
            'T2 [s]': 15.2e-6,
            'RB error': 0.00128,
            'SSRO': {
                'p0given1': 0.0553,
                'p1given0': 0.0212
            }},
        'Q5': {
            'Frequency [Hz]': 4.484e9,
            'T1 [s]': 38.6e-6,
            'T2 [s]': 2.6e-6,
            'RB error': 4.7e-4,
            'SSRO': {
                'p0given1': 0.0507,
                'p1given0': 0.0541
            }},
        'Q6': {
            'Frequency [Hz]': 4.659e9,
            'T1 [s]': 36.9e-6,
            'T2 [s]': 4.7e-6,
            'RB error': 4.7e-4,
            'SSRO': {
                'p0given1': 0.0244,
                'p1given0': 0.0571
            }}
    },
    'CZ IRB errors':{
        'Q0-Q2': 0.0171,
        'Q0-Q3': 0.0167,
        'Q1-Q3': 0.0097,
        'Q1-Q4': 0.0188,
        'Q2-Q5': 0.0212,
        'Q3-Q5': 0.0242,
        'Q3-Q6': 0.0157,
        'Q4-Q6': 0.0124
    },
    'Measurement duration [s]': 1.0e-6,
    'Delay duration [s]': 20e-9,
    'Base temperature [K]': 40e-3
}

In [2]:
from qiskit.quantum_info import Pauli, pauli_basis

In [3]:
pauli_basis_list = pauli_basis(num_qubits=1)

In [4]:
for pauli_idx in range(1, len(pauli_basis_list)):
    print(pauli_basis_list[pauli_idx].to_matrix())

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [5]:
import numpy as np

In [14]:
num_qubits = 1
epsilon_cl = 0.1

p_RB_decay = 1 - (2**num_qubits * epsilon_cl)/(2**num_qubits - 1)
lambda_rate = (2**(2*num_qubits)-1)/2**(2*num_qubits) * (1-p_RB_decay)

kraus_coeff_0 = np.sqrt(1 - lambda_rate)
kraus_coeff = np.sqrt(lambda_rate / (2**(2*num_qubits) - 1))

In [16]:
kraus_coeff

np.float64(0.22360679774997894)

In [11]:
p_RB_decay

0.8666666666666667

In [12]:
lambda_rate

0.12499999999999997

In [ ]:
def depolarization_channel(num_qubits,
                           epsilon_cl):
    kraus_op = []
    pauli_basis_list = pauli_basis(num_qubits=num_qubits)
    
    p_RB_decay = 1 - (2**num_qubits * epsilon_cl)/(2**num_qubits - 1)
    lambda_rate = (2**(2*num_qubits)-1)/2**(2*num_qubits) * (1-p_RB_decay)
    kraus_coeff_0 = np.sqrt(1 - lambda_rate)
    kraus_coeff = np.sqrt(lambda_rate / (2**(2*num_qubits) - 1))
    
    kraus_op.append(kraus_coeff_0 * pauli_basis_list[0].to_matrix())
    for pauli_idx in range(1, len(pauli_basis_list)):
        kraus_op.append(kraus_coeff * pauli_basis_list[pauli_idx].to_matrix())
        
    return kraus_op

In [18]:
kraus_op = depolarization_channel(1, 0.1)

In [ ]:
def depolarization_channel(RB_simple_error):
    kraus_op = []
    decay_prob = (3/2) * RB_simple_error
    kraus_op.append( [[np.sqrt(1 - decay_prob), 0],
                      [0, np.sqrt(1 - decay_prob)]] )
    kraus_op.append( [[0, np.sqrt(decay_prob / 3)],
                      [np.sqrt(decay_prob / 3), 0]] )
    kraus_op.append( [[0, -1j * np.sqrt(decay_prob / 3)],
                      [1j * np.sqrt(decay_prob / 3), 0]] )
    kraus_op.append( [[np.sqrt(decay_prob / 3), 0],
                      [0, -np.sqrt(decay_prob / 3)]] )
    return kraus_op

def depolarization_channel_new(RB_simple_error):
    kraus_op = []
    pauli_basis_list = pauli_basis(num_qubits=1)
    decay_prob = (3/2) * RB_simple_error
    
    kraus_op.append(np.sqrt(1 - decay_prob) * pauli_basis_list[0].to_matrix())
    
    for pauli_idx in range(1, len(pauli_basis_list)):
        kraus_op.append(np.sqrt(decay_prob / 3) * pauli_basis_list[pauli_idx].to_matrix())
        
    return kraus_op

In [ ]:
depolarization_channel(0.005)

In [ ]:
depolarization_channel_new(0.005)

In [ ]:
from qiskit_aer import noise
from scipy.constants import Planck, Boltzmann
from qiskit.quantum_info import Pauli, pauli_basis

def relaxation_channel(temperature, qubit_freq, qubit_T1, time_interval):
    kraus_op = []
    decay_prob = 1 / (1 + np.exp( (-Planck * qubit_freq) / (Boltzmann * temperature) ))
    gamma_rate = 1 - np.exp( - time_interval / qubit_T1)
    kraus_op.append( [[np.sqrt(decay_prob), 0],
                      [0, np.sqrt(decay_prob * (1 - gamma_rate))]] )
    kraus_op.append( [[0, np.sqrt(decay_prob * gamma_rate)],
                      [0, 0]] )
    kraus_op.append( [[np.sqrt((1 - decay_prob)*(1 - gamma_rate)), 0],
                      [0, np.sqrt(1 - decay_prob)]] )
    kraus_op.append( [[0, 0],
                      [np.sqrt(gamma_rate * (1 - decay_prob)), 0]] )
    return kraus_op

def dephasing_channel(qubit_T2, time_interval):
    kraus_op = []
    lambda_rate = 1 - np.exp(- time_interval / qubit_T2)
    kraus_op.append( [[1, 0],
                      [0, np.sqrt(1 - lambda_rate)]] )
    kraus_op.append( [[0, 0],
                      [0, np.sqrt(lambda_rate)]])
    return kraus_op

def depolarization_channel(num_qubits,
                           epsilon_cl):
    kraus_op = []
    pauli_basis_list = pauli_basis(num_qubits=num_qubits)
    
    p_RB_decay = 1 - (2**num_qubits * epsilon_cl)/(2**num_qubits - 1)
    lambda_rate = (2**(2*num_qubits)-1)/2**(2*num_qubits) * (1-p_RB_decay)
    kraus_coeff_0 = np.sqrt(1 - lambda_rate)
    kraus_coeff = np.sqrt(lambda_rate / (2**(2*num_qubits) - 1))
    
    kraus_op.append(kraus_coeff_0 * pauli_basis_list[0].to_matrix())
    for pauli_idx in range(1, len(pauli_basis_list)):
        kraus_op.append(kraus_coeff * pauli_basis_list[pauli_idx].to_matrix())
        
    return kraus_op




def create_noise_model(processor_specs: dict):
    
    qubit_list = list(processor_specs['Qubits'])
    CZ_list = list(processor_specs['CZ IRB errors'])
    
    simulator_noise_model = noise.NoiseModel()
    
    for qubit_idx in range(len(qubit_list)):
        relaxation_noise = noise.kraus_error(relaxation_channel(temperature=processor_specs['Base temperature [K]'],
                                                                qubit_freq=processor_specs[qubit_list[qubit_idx]]['frequency'],
                                                                qubit_T1=processor_specs[qubit_list[qubit_idx]]['T1_eff'],
                                                                time_interval=wait_time))
        
        
        depolarization_noise_x = noise.kraus_error(depolarization_channel(processor_specs[qubit_list[qubit_idx]]['RB_simple_error']))
        readout_error = noise.ReadoutError([[1 - processor_specs[qubit_list[qubit_idx]]['SSRO']['p1given0'],
                                             processor_specs[qubit_list[qubit_idx]]['SSRO']['p1given0']],
                                             [processor_specs[qubit_list[qubit_idx]]['SSRO']['p0given1'],
                                              1 - processor_specs[qubit_list[qubit_idx]]['SSRO']['p0given1']]])
        simulator_noise_model.add_quantum_error(depolarization_noise_x, ['x'], [qubit_idx], warnings=False)
        simulator_noise_model.add_quantum_error(relaxation_noise_wait_time, ['delay'], [qubit_idx], warnings=False)
        simulator_noise_model.add_quantum_error(relaxation_noise_idle_time, ['idle_time'], [qubit_idx], warnings=False)
        simulator_noise_model.add_readout_error(readout_error, [qubit_idx])

    return simulator_noise_model

In [ ]:
num_qubits = 7

wait_time_gate = IGate(label='wait_time')
measurement_time_gate = IGate(label='measurement_time')
idle_time_gate = IGate(label='idle_time')
RO_leakage_gate = PauliGate('III')
RO_leakage_gate.label = 'RO_leakage'

def create_subcircuit(num_qubits,
                      initial_state: str):

    qc = QuantumCircuit(QuantumRegister(num_qubits))

    initial_state = initial_state[::-1]
    for bit_idx in range(len(initial_state)):
        if initial_state[bit_idx] == '1':
            qc.x(bit_idx)
    qc.barrier()

    for qubit in range(num_qubits):
        qc.x(qubit)
    qc.barrier()
    for qubit in range(num_qubits):
        qc.append(wait_time_gate, [qubit])

    # qc.barrier()
    # qc.append(RO_leakage_gate, [1, 3, 4])
    
    qc.measure_all(add_bits = True)
    qc.barrier()
    for qubit in range(num_qubits):
        qc.append(idle_time_gate, [qubit])

    qc.save_statevector()

    return qc

In [ ]:
backend = AerSimulator()

In [ ]:
noise_model = create_noise_model(qubit_dictionary, device_temperature, wait_time, idle_time)

measurement_result = backend.run(circuits=qc,
                                        noise_model=noise_model,
                                        optimization_level = 0,
                                        shots = 1)

measurement_outcome = list(measurement_result.result().get_counts().keys())[0]